# Tensorflow Fruits360-VGG16 Patch Demo

This notebook demonstrates the adversarial patch attack applied on the VGG16 model, as well as adversarial training defenses, and **it is assumed that you have a copy of the Fruits360 dataset and have access to a CUDA-compatible GPU (this demo cannot be run on a typical personal computer)**.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

## Setup: Experiment Name and Fruits360 Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [ ]:
# Import packages from the Python standard library
import importlib.util
import sys
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "fruits360_adversarial_patches"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/dioptra/data/Fruits360"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [ ]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

## Adversarial Patches: Baseline Training

Now, we will train our baseline VGG16 model on the Fruits360 dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.
Once the experiment is finished, we will examine the accuracy results of our model.

In [ ]:
response_vgg16_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        f"-P register_model_name={EXPERIMENT_NAME}_vgg16",
        "-P image_size=224,224,3",
        "-P model_architecture=vgg16",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for VGG16 neural network submitted")
print("")
pprint.pprint(response_vgg16_train)

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [ ]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)

    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response

Now wait for the job to complete before proceeding to next steps.

In [ ]:
response_vgg16_train = wait_until_finished(response_vgg16_train)
print("Training job for VGG16 neural network")
pprint.pprint(response_vgg16_train)

## Checking baseline VGG16 job accuracy

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.
Here we will show the baseline accuracy results from the previous training job.
Please see [Querying the MLFlow Tracking Service](#Querying-the-MLFlow-Tracking-Service) section for more details.

In [ ]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)

    if(job_response['status']=="failed"):
        return {}

    run = mlflow_client.get_run(job_response["mlflowRunId"])

    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])

    return run


results = get_mlflow_results(response_vgg16_train)
pprint.pprint(results.data.metrics)

## Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.
We will also apply the patches over the training set for the adversarial training defense evaluation.

### Patch Generation

The following job will generate the adversarial patches. 
Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [ ]:
# Create Patches
response_vgg16_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/training",
            "-P num_patch_gen_samples=20",
            "-P patch_target=5"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_train["jobId"],
)

print("Patch attack (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_vgg16_patches)
print("")

response_vgg16_patches = get_run_id(response_vgg16_patches)

### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [ ]:
# Deploy Patch attack on training set.
response_deploy_vgg16_patches_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P imagenet_preprocessing=True",
            "-P image_size=224,224,3",
            f"-P data_dir={DATASET_DIR}/training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_training)
print("")

response_deploy_vgg16_patches_training = get_run_id(response_deploy_vgg16_patches_training)

# Deploy Patch attack on test set.
response_deploy_vgg16_patches_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P patch_deployment_method=corrupt"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_testing)
print("")

response_deploy_vgg16_patches_testing = get_run_id(response_deploy_vgg16_patches_testing)

## Patch Attack Evaluation: Baseline VGG16 Model

Now we will run an inference step to check the patch-attacked dataset with our VGG16-trained model.

In [ ]:
# Check patched dataset results
response_infer_vgg16_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P batch_size=512",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_testing["jobId"],
)

print("Patch evaluation (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_infer_vgg16_patch)
print("")

In [ ]:
# Wait for the job to finish
response_infer_vgg16_patch = wait_until_finished(response_infer_vgg16_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_infer_vgg16_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)


We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

# Defense: Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Adversarial Training Defense

We will train a new copy of the VGG16 model on training set that contains adversarial patches.
In doing so, the model learns to ignore the adversarial patches.

In [ ]:
response_patches_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train_on_Fruits360_patched",
    entry_point_kwargs=" ".join(
        [
            f"-P dataset_run_id_testing={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P dataset_run_id_training={response_deploy_vgg16_patches_training['mlflowRunId']}",
            "-P batch_size=256",
            f"-P register_model_name={EXPERIMENT_NAME}_adversarial_patch_vgg16",
            "-P image_size=224,224,3",
            "-P epochs=10",
            f"-P data_dir_testing={DATASET_DIR}/testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_training["jobId"],
)

print("Patch adversarial training (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_patches_adv_training)
print("")

response_patches_adv_training = get_run_id(response_patches_adv_training)

In [ ]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_adversarial_patch_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P batch_size=256",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_patches_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_patches_adv_training= wait_until_finished(response_evaluate_adv_training)
results = get_mlflow_results(response_patches_adv_training)
print("Adversarial Training Results:")
pprint.pprint(results.data.metrics)